## Create the regions

### Parameters

In [ ]:
import numpy as np

DESIRED_REGION_SIZE_KM: np.float64 = 1.0

### Get domain bounding box

In [ ]:
import matplotlib.pyplot as plt
from dataset_objects.bbox import BoundingBox
from remote.remote import Remote

whole_region_bbox: BoundingBox = Remote.get_bounding_box()
whole_region_bbox.summary()

plt.scatter(
    [whole_region_bbox.north_east.longitude, whole_region_bbox.north_west.longitude, whole_region_bbox.south_east.longitude, whole_region_bbox.south_west.longitude],
    [whole_region_bbox.north_east.latitude, whole_region_bbox.north_west.latitude, whole_region_bbox.south_east.latitude, whole_region_bbox.south_west.latitude]
)
plt.show()

### Calculate south-west points

In [ ]:
cells_height:int = round(whole_region_bbox.height / DESIRED_REGION_SIZE_KM)
cells_width:int = round(whole_region_bbox.width / DESIRED_REGION_SIZE_KM)

print(f'Matrix will be {cells_height} cells high and {cells_width} cells wide.')
bases_x = [((x / cells_width) * (whole_region_bbox.south_east.longitude - whole_region_bbox.south_west.longitude)) + whole_region_bbox.south_west.longitude
              for x in range(cells_width) for y in range(cells_height)]
bases_y = [((y / cells_height) * (whole_region_bbox.north_west.latitude - whole_region_bbox.south_west.latitude)) + whole_region_bbox.south_west.latitude
              for x in range(cells_width) for y in range(cells_height)]
plt.scatter(
    bases_x,
    bases_y
)
plt.title("South east coordinate of each cell")
plt.show()

### Create bounds based on south-west points

In [ ]:
from dataset_objects.coordinates import Coordinates

difference_in_latitude = bases_y[1] - bases_y[0]
difference_in_longitude = bases_x[cells_height + 1] - bases_x[0]
print(f'Difference in latitude is {difference_in_latitude:.8f}')
print(f'Difference in longitude is {difference_in_longitude:.8f}')
cells = [
    [ 
        BoundingBox(
            south_west = Coordinates(
                latitude=(y * difference_in_latitude) + whole_region_bbox.south_west.latitude,
                longitude=(x * difference_in_longitude) + whole_region_bbox.south_west.longitude
            ),
            north_east = Coordinates(
                latitude=((y + 1) * difference_in_latitude) + whole_region_bbox.south_west.latitude,
                longitude=((x + 1) * difference_in_longitude) + whole_region_bbox.south_west.longitude
            ),
            x = x,
            y = y
        ) for y in range(cells_height)
    ] for x in range(cells_width)
]

bounds_x = []
bounds_y = []
for x in range(cells_width):
    for y in range(cells_height):
        cell = cells[x][y]
        bounds_x.append(cell.north_east.longitude)
        bounds_x.append(cell.south_east.longitude)
        bounds_x.append(cell.south_west.longitude)
        bounds_x.append(cell.north_west.longitude)

        bounds_y.append(cell.north_east.latitude)
        bounds_y.append(cell.south_east.latitude)
        bounds_y.append(cell.south_west.latitude)
        bounds_y.append(cell.north_west.latitude)

plt.scatter(
    bounds_x,
    bounds_y
)
plt.show()

## Create the dataset shape

### Parameters

In [ ]:

N = 7 # Number of days to look back at
F = 4 # Number of features per region per day
G = 2 # Number of geografical features per region

### Region centers

In [ ]:
import matplotlib.pyplot as plt
from features.geographical.latitude import Latitude
from features.geographical.longitude import Longitude

region_centers = np.zeros(
    shape=(
        cells_width,
        cells_height,
        G
    ),
    dtype=np.float64
)

normalized_region_centers = np.zeros(
    shape=(
        cells_width,
        cells_height,
        G
    ),
    dtype=np.float64
)

normalized_x_for_plot = []
normalized_y_for_plot = []
x_for_plot = []
y_for_plot = []
for x in range(cells_width):
    for y in range(cells_height):
        region_centers[x, y, :] = cells[x][y].center.to_array()
        normalized_region_centers[x][y][0] = Latitude.create_for(cells[x][y].center, whole_region_bbox).normalize()
        normalized_region_centers[x][y][1] = Longitude.create_for(cells[x][y].center, whole_region_bbox).normalize()

        normalized_x_for_plot.append(normalized_region_centers[x][y][1])
        normalized_y_for_plot.append(normalized_region_centers[x][y][0])
        x_for_plot.append(region_centers[x][y][1])
        y_for_plot.append(region_centers[x][y][0])

plt.scatter(normalized_x_for_plot, normalized_y_for_plot)
plt.show()
# plt.scatter(x_for_plot, y_for_plot)
# plt.show()

### Temporal feature shape

In [ ]:
date_range = Remote.get_date_range()
dataset_size = (len(date_range) - N) * cells_width * cells_height

first_input_features = np.zeros(
    shape=(
        dataset_size,
        N,
        F
    ),
    dtype=np.float64
)

### Geographical feature shape

In [ ]:
from remote.remote import Remote
import numpy as np

date_range = Remote.get_date_range()
dataset_size = (len(date_range) - N) * cells_width * cells_height

second_input_features = np.zeros(
    shape=(
        dataset_size,
        2
    ),
    dtype=np.float64
)

### Output shape

In [ ]:
from remote.remote import Remote
import numpy as np

date_range = Remote.get_date_range()
dataset_size = (len(date_range) - N) * cells_width * cells_height

outputs = np.zeros(
    shape=(dataset_size),
    dtype=np.float64
)

## Create the dataset

### Define last N days queue shape

In [ ]:
region_queues = np.zeros(shape=(
    cells_width,
    cells_height,
    len(date_range),
    F
))

### Pre-populate queue with first N days

In [ ]:
from features.factory import Factory

# Iterate initial N dates to populate the queues
for index, date in enumerate(date_range[:N]):

    weather_features = Factory.get_weather_features(date)
    risk_features, _ = Factory.get_risk_features(date, cells, whole_region_bbox, cells_width, cells_height, 2, DESIRED_REGION_SIZE_KM)

    for xi in range(cells_width):
        for yi in range(cells_height):
            region_features = risk_features[xi][yi]

            for z in range(len(region_features)):
                region_queues[xi][yi][index][z] = region_features[z]

            for z in range(len(weather_features)):
                region_queues[xi][yi][index][z + len(region_features)] = weather_features[z]

### Populate dataset

In [ ]:
global_index = 0
queues_index = N
accidents_per_day = []
for date in date_range[N:]:

    print(f'Processing date {date}', end='\r')
    weather_features = Factory.get_weather_features(date)
    risk_features, total_acc = Factory.get_risk_features(date, cells, whole_region_bbox, cells_width, cells_height, 2, DESIRED_REGION_SIZE_KM)
    accidents_per_day.append(total_acc)
    for xi in range(cells_width):
        for yi in range(cells_height):

            region_features = risk_features[xi][yi]
            expected_risk_index = risk_features[xi][yi][0]

            first_input_features[global_index][:] = region_queues[xi][yi][-N:]
            second_input_features[global_index][:] = normalized_region_centers[xi][yi]
            outputs[global_index] = expected_risk_index

            global_index += 1
            for z in range(len(region_features)):
                region_queues[xi][yi][queues_index][z] = region_features[z]

            for z in range(len(weather_features)):
                region_queues[xi][yi][queues_index][z + len(region_features)] = weather_features[z]
            
    queues_index += 1


In [ ]:
print(f'Average accidents per day is {np.average(accidents_per_day)}')
print(f'Max accidents per day is {np.max(accidents_per_day)}')
print(f'Min accidents per day is {np.min(accidents_per_day)}')

In [ ]:
print(f'Max index is {np.max(outputs)}')
print(f'Min index is {np.min(outputs)}')
print(f'Avg index is {np.average(outputs)}')
print(f'Median index is {np.median(outputs)}')

In [ ]:
zeros_leading_to_something = 0
for i, feature_set in enumerate(first_input_features):
    label = outputs[i]
    has_non_zero = False
    for j in range(7):
        if (feature_set[j][0] != 0):
            has_non_zero = True
    if not has_non_zero and label != 0:
        zeros_leading_to_something += 1
print(f'A bunch of zero indices leading to a non-zero index happened {zeros_leading_to_something} times')
print(f'For reference, the dataset is {len(first_input_features)} inputs big')

## Save the dataset

In [ ]:
import os
import pickle as pkl

FOLDER = "data/"
INPUTS_1_FILE = FOLDER + "inputs_1.pkl"
INPUTS_2_FILE = FOLDER + "inputs_2.pkl"
OUTPUTS_FILE = FOLDER + "outputs.pkl"
CENTERS_FILE = FOLDER + "centers.pkl"
REGIONS_FILE = FOLDER + "regions.pkl"
N_CENTERS_FILE = FOLDER + "normal_centers.pkl"
SHAPE_FILE = FOLDER + "shape.pkl"

if not os.path.exists(INPUTS_1_FILE):
    with open(INPUTS_1_FILE, 'wb') as h:
        pkl.dump(first_input_features, h)

if not os.path.exists(INPUTS_2_FILE):
    with open(INPUTS_2_FILE, 'wb') as h:
        pkl.dump(second_input_features, h)

if not os.path.exists(OUTPUTS_FILE):
    with open(OUTPUTS_FILE, 'wb') as h:
        pkl.dump(outputs, h)

if not os.path.exists(CENTERS_FILE):
    with open(CENTERS_FILE, 'wb') as h:
        pkl.dump(region_centers, h)

if not os.path.exists(N_CENTERS_FILE):
    with open(N_CENTERS_FILE, 'wb') as h:
        pkl.dump(normalized_region_centers, h)

if not os.path.exists(REGIONS_FILE):
    with open(REGIONS_FILE, 'wb') as h:
        pkl.dump(cells, h)

if not os.path.exists(SHAPE_FILE):
    with open(SHAPE_FILE, 'wb') as h:
        pkl.dump([cells_width, cells_height, G, N, F, dataset_size], h)